In [2]:
import pandas as pd
import nltk
import string

# Baixa os recursos necessários do NLTK
nltk.download('punkt')        # Tokenizador pré-treinado
nltk.download('stopwords')    # Lista de stopwords (palavras irrelevantes) em vários idiomas
nltk.download('wordnet')      # Base de dados semântica usada na lematização (em inglês)

# Importa componentes do NLTK
from nltk.corpus import stopwords  # Lista de palavras irrelevantes para remoção
from nltk.stem import SnowballStemmer, RSLPStemmer, PorterStemmer, WordNetLemmatizer  # Stemmers e lematizador
from nltk.tokenize import TweetTokenizer  # Tokenizador ideal para textos informais (como tweets, emojis, hashtags)

from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     Basic Constraints of CA cert not marked critical
[nltk_data]     (_ssl.c:1028)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     Basic Constraints of CA cert not marked critical
[nltk_data]     (_ssl.c:1028)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     Basic Constraints of CA cert not marked critical
[nltk_data]     (_ssl.c:1028)>


In [ ]:
df = pd.read_csv('database/aa_dataset-tickets-multi-lang-5-2-50-version.csv')

In [ ]:
df.head(5)

In [ ]:
# Verificando quais idiomas estão os emails 
print("Idiomas: " + ', '.join(df['language'].unique().astype(str)))
# (DE- Alemão, EN- Inglês)

# Verificando quais categorias existem
print("Categorias: " + ", " .join(df['type'].unique().astype(str)))

In [ ]:
# Filtrando os emaisl apenas na lingua inglesa
english_df = df[df['language'] == "en"]

# Removendo colunas não utilizadas
english_df = english_df.drop(columns= ["answer", "queue", "priority", "language", "version", "tag_1", "tag_2", "tag_3", "tag_4", "tag_5", "tag_6", "tag_7", "tag_8"])

In [ ]:
english_df

# Pré-processamento de texto

In [ ]:
# Tokenizador e ferramentas
tokenizador = TweetTokenizer()
stemmer_en = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stopwords_en = set(stopwords.words('english'))

def pre_processamento(texto_en):
    # Garantir que o texto seja string
    texto_en = str(texto_en)

    # Tokenização
    tokens_en = tokenizador.tokenize(texto_en)

    # Tokens sem pontuação
    tokens_en_sem_pontuacao = [t for t in tokens_en if t not in string.punctuation]

    # Stopwords
    tokens_sem_stopwords_en = [t for t in tokens_en_sem_pontuacao if t.lower() not in stopwords_en]

    # Stemming
    stems_en = [stemmer_en.stem(t) for t in tokens_sem_stopwords_en]

    # Lematização
    lemmas_en = [lemmatizer.lemmatize(t.lower()) for t in tokens_sem_stopwords_en]

    return lemmas_en  # ou tokens_sem_stopwords_en, ou stems_en, conforme o que quiser usar

In [ ]:
english_df['body_processado'] = english_df['body'].apply(pre_processamento)

In [ ]:
english_df

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Junta todos os textos em uma única string
texto = ' '.join(english_df['body_processado'].astype(str))

# Cria a nuvem de palavras
nuvem = WordCloud(
    background_color='white',
    stopwords=stopwords_en,
    width=800,
    height=400,
    colormap='viridis'
).generate(texto)

# Exibe a nuvem
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.imshow(nuvem, interpolation='bilinear')
plt.axis('off')
plt.title("Nuvem de Palavras")
plt.show()

# Extração de features

## TF-IDF
Transforma texto em vetores numéricos com base na frequência relativa das palavras no documento.

In [ ]:
# Converta para string, caso necessário
corpus = english_df['body'].astype(str)

# Inicializa o vetorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')  # Pode incluir mais params como ngram_range, max_features, etc.
X_tfidf = vectorizer.fit_transform(corpus)

# Supondo que sua coluna 'body' já esteja pré-processada e unida como texto
corpus = english_df['body'].astype(str)

# Criar o vetor TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)  # Limite de features, pode ajustar
tfidf_matrix = vectorizer.fit_transform(corpus)

# tfidf_matrix é uma matriz esparsa (documentos x palavras)
print("Shape:", tfidf_matrix.shape)

#  transformar em um DataFrame:
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df_tfidf